In [ ]:
!python -m spacy download pt_core_news_sm

In [1]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

spacy.prefer_gpu()

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("modelo_skipgram.txt")
artigo_treino = pd.read_csv("../introducao-word-embedding/treino.csv")
artigo_teste = pd.read_csv("../introducao-word-embedding/teste.csv")

In [2]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [3]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 1.35496333e-01  2.48119560e+00  1.35839716e+00  5.15865952e-01
   4.14586097e+00 -8.06703851e-01 -1.07618248e+00  3.18539590e+00
   1.33622754e+00 -1.87997244e+00 -1.01188534e+00 -5.28258681e-01
  -2.66777650e-01  2.56107658e-01 -4.11424503e-01 -1.21356970e+00
   1.72387844e+00  4.60876465e-01  1.66882755e+00  9.06709813e-01
  -1.71362711e+00  1.45900123e-01  1.05301522e-01  3.15188468e+00
   2.80815399e-01 -5.61452629e-01  2.42183506e-02  2.60848173e+00
  -5.14426664e-01  2.06357020e+00 -4.45397642e-01 -1.26932591e-01
   1.68718985e+00 -7.20640633e-01 -1.94688791e+00 -1.18843151e+00
  -3.77514176e-02 -1.34002839e+00 -5.38701827e-01  8.48240623e-01
  -2.54380751e-01 -7.16149107e-01 -3.40202525e-02 -4.14664600e-01
  -7.58426458e-01 -6.78785816e-01  1.01885460e+00  5.34720957e-01
   5.68616301e-01 -3.40298191e-01 -4.93754748e-01 -2.30744720e+00
  -6.13884859e-01 -1.49822336e+00  1.72477363e+00 -6.01086423e-01
   1.45934559e+00 -7.78351218e-01  2.92060822e-01  1.16249223e+00
 

In [4]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)
